In [1]:
#!pip install transformers datasets accelerate huggingface_hub

In [2]:
# !pip install -U trl

In [3]:
import shutil

# shutil.rmtree("./mlruns")
# shutil.rmtree("./tmp")
# shutil.rmtree("./qwen-0.5b-finetuned-lora")

In [4]:
import os
import json
from getpass import getpass
from huggingface_hub import login

In [5]:
token = getpass(prompt="enter huggingface token")
login(token=token)

enter huggingface token ········


In [6]:
dir_name = "data"

if os.path.isfile(dir_name + '/_topics.json'):
    with open(dir_name + '/_topics.json') as t:
        topics = json.load(t)

    topics = topics.keys()
    tfiles = [dir_name + f"""/{"-".join(t.lower().split(' '))}.json""" for t in topics]
    tfiles = [tf for tf in tfiles if os.path.isfile(tf)]

    data = []
    for tf in tfiles:
        with open(tf) as t:
            data.extend(json.load(t))

    print(f'there are {len(data)} rows in the dataset')

there are 388 rows in the dataset


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

In [8]:
dataset = load_dataset("json", data_files=tfiles)

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

In [9]:
print(dataset["train"][0]["answer"])

Rare earth metals are integral to various green technology applications, with specific elements serving crucial functions. Wind turbines utilize neodymium and dysprosium in their permanent magnets, enabling more efficient electricity generation without mechanical gearboxes <document-1>. These same magnetic properties make rare earths essential in electric vehicle motors, with each EV requiring approximately 1kg of these materials <document-1>. Energy-efficient LED lighting relies on europium, terbium, and yttrium, which significantly reduce energy consumption compared to traditional lighting options <document-1>.

Energy storage represents another key application, with lanthanum being used in NiMH batteries that support hybrid vehicles and grid-scale renewable energy systems <document-2>. For emission reduction, cerium is incorporated into catalytic converters <document-2>. In specialized renewable energy applications where durability in extreme temperatures is required, samarium-cobal

In [10]:
dataset = dataset["train"].train_test_split(test_size=0.1)

In [11]:
PROMPT = """
You are a factual answer generator that must respond only with information explicitly found in the provided retrieved documents. Do not include any external knowledge or assumptions. When constructing your answer, follow these guidelines:

1. Strict Document Reliance:
   Base all responses exclusively on the facts and data contained in the retrieved documents. If a piece of information is used, it must come from one of these documents.

2. Citation Style:
   For every factual statement or claim you include, append the corresponding document reference in angle brackets (e.g., <document-1>, <document-2>) immediately after the relevant sentence or clause.

3. Structured, Factual Language:
   Use clear, concise, and informative language. Model your responses after the following example:

   Rare earth metals are integral to various green technology applications, with specific elements serving crucial functions. Wind turbines utilize neodymium and dysprosium in their permanent magnets, enabling more efficient electricity generation without mechanical gearboxes <document-1>. These same magnetic properties make rare earths essential in electric vehicle motors, with each EV requiring approximately 1kg of these materials <document-1>.

   Energy-efficient LED lighting relies on europium, terbium, and yttrium, which significantly reduce energy consumption compared to traditional lighting options <document-1>.

   Energy storage represents another key application, with lanthanum being used in NiMH batteries that support hybrid vehicles and grid-scale renewable energy systems <document-2>. For emission reduction, cerium is incorporated into catalytic converters <document-2>. In specialized renewable energy applications where durability in extreme temperatures is required, samarium-cobalt magnets provide high efficiency for electric motors and generators <document-2>. Wind turbines specifically require substantial amounts of rare earth elements—approximately 600kg per turbine—with these materials enabling electricity generation in lower wind speeds while improving overall efficiency <document-3>. The rare earth elements also contribute to advanced photovoltaic systems, where lanthanum and cerium compounds are used in polishing powders for precision optical lenses, improving solar energy capture efficiency by 15-20% compared to conventional systems <document-4>.

4. Answering Questions:
   - If the user’s question can be fully answered by the retrieved documents, synthesize the relevant information and include the proper document citations.
   - If the question requires details that are not contained within the documents, respond that the available documents do not provide sufficient information.

5. Synthesis and Clarity:
   When combining information from multiple documents, ensure your answer remains clear and logically structured, with citations indicating which facts come from which documents.

Documents:
{documents}
"""


In [12]:
def preprocess(tokenizer):
    def f(row):
        output_texts = []
    
        for i in range(len(row['question'])):
            messages = [
                {"role": "system", "content": PROMPT.format(documents=row["documents"][i])},
                {"role": "user", "content": row["question"][i]},
                {"role": "assistant", "content": row["answer"][i]}
            ]
            output_texts.append(tokenizer.apply_chat_template(messages, tokenize=False))
            
        return output_texts
    return f

In [13]:
from transformers import BitsAndBytesConfig

# model_id = "meta-llama/Llama-3.2-1B-Instruct"

model_id = "HuggingFaceTB/SmolLM2-360M-Instruct"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id) #, quantization_config=bnb_config)

In [14]:
from peft import LoraConfig, get_peft_model

# Define LoRA Configuration
lora_config = LoraConfig(
    r=4,
    lora_alpha=32,
    # lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 819,200 || all params: 362,640,320 || trainable%: 0.2259


In [15]:
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(49153, 960, padding_idx=2)

In [16]:
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

# template for llama
# response_template = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

# template for Qwen (ChatML)
response_template = "<|im_start|>assistant\n"
# print(response_template)
response_token_ids = tokenizer.encode(response_template, add_special_tokens=False)
collator = DataCollatorForCompletionOnlyLM(response_template=response_token_ids, tokenizer=tokenizer)

In [22]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    args=SFTConfig(output_dir="./tmp",
                   max_seq_length=2048,
                   eval_strategy="steps",
                   eval_steps=10,
                   logging_steps=10,
                   num_train_epochs=27,
                   do_eval=True,
                   per_device_train_batch_size=2,
                   report_to="none"),
    formatting_func=preprocess(tokenizer),
    data_collator=collator,
)

trainer.train()

Applying formatting function to train dataset:   0%|          | 0/349 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/349 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/349 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/349 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/349 [00:00<?, ? examples/s]

...

Truncating eval dataset:   0%|          | 0/39 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss
10,0.957000,0.927922
20,1.013800,0.925575
30,0.980200,0.923292
40,0.921600,0.920947
50,0.853100,0.918905
60,0.957900,0.916887
70,0.881900,0.914929
80,1.055300,0.912829
90,0.902400,0.910590
100,0.939900,0.908517


/usr/local/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

TrainOutput(global_step=4725, training_loss=0.8015181865893974, metrics={'train_runtime': 6515.077, 'train_samples_per_second': 1.446, 'train_steps_per_second': 0.725, 'total_flos': 2.53686813353472e+16, 'train_loss': 0.8015181865893974})

In [23]:
model.save_pretrained("./smol-finetuned-lora")
tokenizer.save_pretrained("./smol-finetuned-lora")

/usr/local/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('./smol-finetuned-lora/tokenizer_config.json',
 './smol-finetuned-lora/special_tokens_map.json',
 './smol-finetuned-lora/vocab.json',
 './smol-finetuned-lora/merges.txt',
 './smol-finetuned-lora/added_tokens.json',
 './smol-finetuned-lora/tokenizer.json')

In [24]:
documents = """
Document 1: But it’s been thirty years and vegans are still less than 10% of the population.If you genuinely care about animal suffering, you have to admit that, and say, ‘what else can we do to ease animal suffering?’”After Singer’s talk, I began thinking through the consequences of his morality.A question occurred to me: “Should we also stop animals from eating each other?” I was sure others had made such arguments as *reductio ad absurdums* of vegetarianism, but I thought I might be the first to be genuinely interested in it from a moral perspective.
---
Document 2: They didn’t even pretend to make an argument; it was entirely mockery.Watching this, I couldn’t help but realize there was a powerful logical argument at the core of the animal rights groups: animals should be treated much the same way humans are — their lives should be respected, their pain minimized, etc.Make this one simple change to your system of morality and everything else falls into place.
---
Document 3: But I praise people for the good things they do and condemn them for the bad ones.”A final question raised the incrementalism versus revolutionism debate common to all left-wing social movements.Should one really worry about animal treatment when the animals were still going to be killed?Pinger said the answer was undoubtedly yes.
---
Document 4: They didn’t even pretend to make an argument; it was entirely mockery.Watching this, I couldn’t help but realize there was a powerful logical argument at the core of the animal rights groups: animals should be treated much the same way humans are — their lives should be respected, their pain minimized, etc.Make this one simple change to your system of morality and everything else falls into place.
---
Document 5: But I praise people for the good things they do and condemn them for the bad ones.”A final question raised the incrementalism versus revolutionism debate common to all left-wing social movements.Should one really worry about animal treatment when the animals were still going to be killed?Pinger said the answer was undoubtedly yes.
---
Document 6: But it’s been thirty years and vegans are still less than 10% of the population.If you genuinely care about animal suffering, you have to admit that, and say, ‘what else can we do to ease animal suffering?’”After Singer’s talk, I began thinking through the consequences of his morality.A question occurred to me: “Should we also stop animals from eating each other?” I was sure others had made such arguments as *reductio ad absurdums* of vegetarianism, but I thought I might be the first to be genuinely interested in it from a moral perspective.
""".strip()

question = "Should animals be treated like humans?"

In [29]:
from peft import PeftModel
from transformers import pipeline

pipeline_model = AutoModelForCausalLM.from_pretrained(model_id) #, quantization_config=bnb_config)
pipeline_model.resize_token_embeddings(len(tokenizer))
pipeline_model = PeftModel.from_pretrained(pipeline_model, "./smol-finetuned-lora")

generator = pipeline("text-generation", model=pipeline_model, tokenizer=tokenizer, max_length=1536)

messages = [
    {"role": "system", "content": PROMPT.format(documents=documents)},
    {"role": "user", "content": question},
]
response = generator(messages, repetition_penalty=1.01)
print(len(response))
print()
print()
print(documents)
print()
print()
print(response[0]['generated_text'][-1]['content'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCa

1


Document 1: But it’s been thirty years and vegans are still less than 10% of the population.If you genuinely care about animal suffering, you have to admit that, and say, ‘what else can we do to ease animal suffering?’”After Singer’s talk, I began thinking through the consequences of his morality.A question occurred to me: “Should we also stop animals from eating each other?” I was sure others had made such arguments as *reductio ad absurdums* of vegetarianism, but I thought I might be the first to be genuinely interested in it from a moral perspective.
---
Document 2: They didn’t even pretend to make an argument; it was entirely mockery.Watching this, I couldn’t help but realize there was a powerful logical argument at the core of the animal rights groups: animals should be treated much the same way humans are — their lives should be respected, their pain minimized, etc.Make this one simple change to your system of morality and everything else falls into place.
---
Document 3: But

In [30]:
shutil.make_archive('./finetuned_model', 'zip', './smol-finetuned-lora')

'/users/anefufav/finetuned_model.zip.zip'